##Wikipedia API

#To Do

1. How do get the concepts from Concept Insights?

2. Do we need to train to answer question #1?

3. If so, how do we train?

4. If we don't need to train, go back to question #1?

###Alexander's Notes

We should be piping the user's input directly into Concept Expansion
build a list of related terms, then piping that list into Concept Insight
in order to collect conceptual context (insight) about terms in that list. 

In [1]:
from wikipedia import Wikipedia
from wiki2plain import Wiki2Plain
import numpy as np

In [2]:
import ast

In [3]:
# print_function turns print into a python 3 function 
# this enables python to print funky text i.e. f\xn --> fun
from __future__ import (division,
                        absolute_import,
                        print_function,
                        unicode_literals
                        )

In [4]:
subjects = ["Calculus", 
            "Trigonometry", 
            "Geometry",
            "Differential Equations", 
            "Integrals", 
            "Unit Circle", 
            "Sine", 
            "Cosine", 
            "pythagorean theorem"]

In [ ]:
import os.path

def scrape_wiki(path,topic):
    '''
    Get all text from wiki page based on the topic entered.
    '''
    lang = 'en' # standard english wiki pages
    wiki = Wikipedia(lang)
    
    
    fname = path + topic + '.txt'
    
    if not os.path.isfile(fname):    
        try:
            raw = wiki.article(s)
        except:
            raw = None

        if raw:
            wiki2plain = Wiki2Plain(raw)
            content = wiki2plain.text

            with open(fname,'w') as handle:
                handle.write(content) 

In [ ]:
path = "data/"
 for subject in subjects:
    scrape_wiki(path,subject)

In [5]:
f = open(path + subjects[-1] + '.txt','r')

In [6]:
print(f.read())

In mathematics, the Pythagorean theorem, also known as Pythagoras' theorem, is a relation in Euclidean geometry among the three sides of a right triangle. It states that the square of the hypotenuse (the side opposite the right angle) is equal to the sum of the squares of the other two sides. The theorem can be written as an equation relating the lengths of the sides a, b and c, often called the "Pythagorean equation":
:a^2 + b^2 = c^2 ,
where c represents the length of the hypotenuse and a and b the lengths of the triangle's other two sides.

Although it is often argued that knowledge of the theorem predates him, the theorem is named after the ancient Greek mathematician Pythagoras ( 570 –  495 BC) as it is he who, by tradition, is credited with its first recorded proof.<!--
 --> <!--
 -->There is some evidence that Babylonian mathematicians understood the formula, although little of it indicates an application within a mathematical framework.<!--
 --><!--
 --> <!--
 -->Mesopotamian, 

##Concept Insights API

##Build Corpus

In [5]:
import json
import requests

In [9]:
# the ConceptInsights api from watson_developer_cloud is difficult to work with
# so I've been using request instead
from watson_developer_cloud import ConceptInsightsV2 as ConceptInsights
from pprint import pprint

In [6]:
# get account_id
base_url = "https://watson-api-explorer.mybluemix.net/concept-insights/api"
account_url = 'https://watson-api-explorer.mybluemix.net/concept-insights/api/v2/accounts'
auth=('82a698f0-8d8c-42f9-9c7d-1e4b97eebd52','50KJG6jdgZ6h')

r = requests.get(account_url, auth=auth)

In [7]:
# display account id
print(r.content)

{"accounts":[{"account_id":"liet4blerinu2"}]}


In [56]:
# create copus called 'math'
# I had a very hard time trying to get this code to work - hours lost ! 
# It still doesn't work. Maybe I should be using '.post' instead of '.put' ...?

# create_corp = "https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math"
# payload = '{"access":"private","users":[{"uid":"82a698f0-8d8c-42f9-9c7d-1e4b97eebd52","permission":"AdminReadWrite"}]}'
# r2 = requests.put(create_corp,
#                   params = payload,
#                   auth=auth)
# r2.status_code

400

In [10]:
# create corpus called 'math_corp'
# this code only needs to run once -- otherwise 409 error will occur

!curl -D - -X PUT -u "82a698f0-8d8c-42f9-9c7d-1e4b97eebd52":'50KJG6jdgZ6h' \
    -d  '{"access":"private","users":[{"uid":"82a698f0-8d8c-42f9-9c7d-1e4b97eebd52","permission":"AdminReadWrite"}]}'\
    "https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp"














{"error":"corpus already exists","url":"/api/v2/corpora/liet4blerinu2/math_corp"}


In [24]:
# get available corpora
available_corpora = 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2'

r = requests.get(available_corpora, auth=auth)
r.status_code

200

In [25]:
# display available corpora and it's parameters
ast.literal_eval(r.content)

{'corpora': [{'access': 'private',
   'id': '/corpora/liet4blerinu2/math_corp',
   'users': [{'account_id': '', 'permission': 'ReadWriteAdmin'},
    {'account_id': 'liet4blerinu2', 'permission': 'ReadWriteAdmin'}]}]}

In [18]:
# DELETE /v2/corpora/{account_id}/{corpus} Deletes a corpus by ID
url = 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp2'
requests.delete(url)

<Response [401]>

In [22]:
# DELETE /v2/corpora/{account_id}/{corpus} Deletes a corpus by ID
!curl -u "82a698f0-8d8c-42f9-9c7d-1e4b97eebd52":'50KJG6jdgZ6h' \
        -X DELETE 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp2'

In [48]:
# upload file to corpus
url = 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp/documents/testDocument'
path ="data/"
for subject in subjects:
    url = 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp/documents/' + subject

    payload = {"label": subject,
               "parts": [{"data": open(path + subject + '.txt','rb').read(),
                          "name": "HTML part",
                          "content-type": "text/html"},
                         {"data": "Part of test document pure text."
                          ,"name": "Text part",
                          "content-type": "text/plain"}],
               "user_fields": {"myField1": "my meta-data"}}

    r2 = requests.post(url = url,
                      data = json.dumps(payload),
                      auth=auth)
    print(r2.status_code)

201
201
201
201
201
201
201
201
201


### retrive document id's

In [ ]:
# GET /v2/corpora/{account_id}/{corpus}/documents
url = 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp/documents'
r = requests.get(url,auth=auth)

print (r.status_code)

In [31]:
ast.literal_eval(r.content)

{'documents': ['/corpora/liet4blerinu2/math_corp/documents/Calculus',
  '/corpora/liet4blerinu2/math_corp/documents/Trigonometry',
  '/corpora/liet4blerinu2/math_corp/documents/Geometry',
  '/corpora/liet4blerinu2/math_corp/documents/Differential%20Equations',
  '/corpora/liet4blerinu2/math_corp/documents/Integrals',
  '/corpora/liet4blerinu2/math_corp/documents/Unit%20Circle',
  '/corpora/liet4blerinu2/math_corp/documents/Sine',
  '/corpora/liet4blerinu2/math_corp/documents/Cosine',
  '/corpora/liet4blerinu2/math_corp/documents/pythagorean%20theorem']}

In [28]:
# get content of document
url = 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp/documents/Trigonometry'
r = requests.get(url,auth=auth)

print (r.status_code)

ast.literal_eval(r.content)

200


{'id': '/corpora/liet4blerinu2/math_corp/documents/Trigonometry',
 'label': 'Trigonometry',
 'last_modified': '2015-12-01T23:36:19.457Z',
 'parts': [{'content-type': 'text/html',
   'data': 'Trigonometry (from Greek trig\xc5\x8dnon, "triangle" and metron, "measure") is a branch of mathematics that studies relationships involving lengths and angles of triangles.  The field emerged in the Hellenistic world during the 3rd century BC from applications of geometry to astronomical studies.\n\nThe 3rd-century astronomers first noted that the lengths of the sides of a right-angle triangle and the angles between those sides have fixed relationships: that is, if at least the length of one side and the value of one angle is known, then all other angles and lengths can be determined algorithmically. These calculations soon came to be defined as the trigonometric functions and today are pervasive in both pure and applied mathematics: fundamental methods of analysis such as the Fourier transform, fo

In [29]:
# delete document from corpus by id
# DELETE /v2/corpora/{account_id}/{corpus}/documents/{document}

! curl -u "82a698f0-8d8c-42f9-9c7d-1e4b97eebd52":'50KJG6jdgZ6h'\
       -X DELETE 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp/documents/testDocument'

### stats on concepts present in corpora 

In [94]:
# GET /v2/corpora/{account_id}/{corpus}/stats
url = 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp/stats'
r = requests.get(url,auth=auth)
r.status_code

200

In [95]:
# stats on concepts present in corpora 
ast.literal_eval(r.content)

{'id': '/corpora/liet4blerinu2/math_corp',
 'last_updated': '2015-12-03T05:35:35.883Z',
 'top_tags': {'corpus_tags_histogram': {'1': 863,
   '10': 13,
   '11': 15,
   '112': 1,
   '113': 1,
   '117': 1,
   '12': 12,
   '13': 12,
   '14': 2,
   '141': 1,
   '145': 1,
   '15': 8,
   '150': 1,
   '152': 1,
   '155': 1,
   '16': 7,
   '160': 1,
   '17': 4,
   '178': 1,
   '18': 4,
   '189': 1,
   '19': 3,
   '195': 1,
   '2': 282,
   '20': 4,
   '21': 1,
   '22': 3,
   '24': 1,
   '25': 3,
   '26': 1,
   '27': 2,
   '29': 5,
   '3': 120,
   '31': 1,
   '32': 4,
   '325': 1,
   '33': 1,
   '34': 1,
   '35': 2,
   '36': 1,
   '37': 1,
   '373': 1,
   '38': 2,
   '39': 2,
   '4': 88,
   '40': 3,
   '41': 1,
   '42': 1,
   '45': 1,
   '47': 1,
   '48': 1,
   '5': 43,
   '51': 1,
   '54': 1,
   '55': 1,
   '56': 3,
   '57': 1,
   '59': 1,
   '6': 30,
   '60': 1,
   '7': 27,
   '71': 1,
   '73': 1,
   '8': 21,
   '87': 1,
   '9': 20,
   '92': 1},
  'document_length_histogram': {'101900': 1,
   '

###graph id

In [80]:
url = 'https://gateway.watsonplatform.net/concept-insights/api/v2/graphs'
#url = "https://watson-api-explorer.mybluemix.net/concept-insights/api/v2/graphs"
r = requests.get(url ,auth=auth)
r.status_code

200

In [81]:
ast.literal_eval(r.content)

{'graphs': ['/graphs/wikipedia/en-20120601', '/graphs/wikipedia/en-latest']}

###label search

In [85]:
# GET /v2/graphs/{account_id}/{graph}/label_search
!curl -u '82a698f0-8d8c-42f9-9c7d-1e4b97eebd52':'50KJG6jdgZ6h''https://gateway.watsonplatform.net/concept-insights/api/v2/graphs/liet4blerinu2/wikipedia/en-latest/label_search?query=math'

curl: no URL specified!
curl: try 'curl --help' or 'curl --manual' for more information


In [194]:
url = 'https://gateway.watsonplatform.net/concept-insights/api'+\
 '/v2/graphs/wikipedia/en-20120601/label_search?query=quadratic'
r = requests.get(url ,auth=auth)
r.status_code

200

In [195]:
ast.literal_eval(r.content)

{'matches': [{'id': '/graphs/wikipedia/en-20120601/concepts/Quadratic',
   'label': 'Quadratic'}]}

In [158]:
#GET /v2/corpora/{account_id}/{corpus}/conceptual_search
url = 'https://gateway.watsonplatform.net/concept-insights/api'+\
 '/v2/corpora/liet4blerinu2/math_corp/conceptual_search?concept_fields=["sine"]'
r  = requests.get(url, auth=auth)
r.status_code

400

In [159]:
r.content #"/graphs/wikipedia/en-20120601/concepts/Slope

'{"error":"missing ids parameter","url":"/api/v2/corpora/liet4blerinu2/math_corp/conceptual_search"}\n'

###Concepts related to a specific article

In [160]:
#GET /v2/corpora/{account_id}/{corpus}/documents/{document}
# Calculus
url = 'https://gateway.watsonplatform.net/concept-insights/api'+\
      '/v2/corpora/liet4blerinu2/math_corp/documents/Calculus/related_concepts'
r  = requests.get(url, auth=auth)
r.status_code

200

In [161]:
ast.literal_eval(r.content)

{'concepts': [{'concept': {'id': '/graphs/wikipedia/en-20120601/concepts/Calculus',
    'label': 'Calculus'},
   'score': 0.99578935},
  {'concept': {'id': '/graphs/wikipedia/en-20120601/concepts/Infinitesimal',
    'label': 'Infinitesimal'},
   'score': 0.98576385},
  {'concept': {'id': '/graphs/wikipedia/en-20120601/concepts/Derivative',
    'label': 'Derivative'},
   'score': 0.98494786},
  {'concept': {'id': '/graphs/wikipedia/en-20120601/concepts/Integral',
    'label': 'Integral'},
   'score': 0.9829364},
  {'concept': {'id': '/graphs/wikipedia/en-20120601/concepts/Limit_(mathematics)',
    'label': 'Limit (mathematics)'},
   'score': 0.98230964},
  {'concept': {'id': '/graphs/wikipedia/en-20120601/concepts/Gottfried_Leibniz',
    'label': 'Gottfried Leibniz'},
   'score': 0.9816452},
  {'concept': {'id': '/graphs/wikipedia/en-20120601/concepts/Indian_mathematics',
    'label': 'Indian mathematics'},
   'score': 0.9806901},
  {'concept': {'id': '/graphs/wikipedia/en-20120601/conc

In [110]:
url = 'https://gateway.watsonplatform.net/concept-insights/api'+\
      '/v2/corpora/liet4blerinu2/math_corp/documents/Sine/related_concepts'
r  = requests.get(url, auth=auth)
r.status_code

200

In [111]:
ast.literal_eval(r.content)

{'concepts': [{'concept': {'id': '/graphs/wikipedia/en-20120601/concepts/Pi',
    'label': 'Pi'},
   'score': 0.9944448},
  {'concept': {'id': '/graphs/wikipedia/en-20120601/concepts/Sine',
    'label': 'Sine'},
   'score': 0.9935893},
  {'concept': {'id': '/graphs/wikipedia/en-20120601/concepts/Square_root',
    'label': 'Square root'},
   'score': 0.9923213},
  {'concept': {'id': '/graphs/wikipedia/en-20120601/concepts/Angle',
    'label': 'Angle'},
   'score': 0.98779887},
  {'concept': {'id': '/graphs/wikipedia/en-20120601/concepts/Trigonometric_functions',
    'label': 'Trigonometric functions'},
   'score': 0.985663},
  {'concept': {'id': '/graphs/wikipedia/en-20120601/concepts/Bracket',
    'label': 'Bracket'},
   'score': 0.9829014},
  {'concept': {'id': '/graphs/wikipedia/en-20120601/concepts/Radian',
    'label': 'Radian'},
   'score': 0.9819788},
  {'concept': {'id': '/graphs/wikipedia/en-20120601/concepts/Plus_and_minus_signs',
    'label': 'Plus and minus signs'},
   'scor

##How Related is Radian (Concept) to Calculus (Document)

In [203]:
# GET /v2/corpora/{account_id}/{corpus}/documents/{document}/relation_scores
# concept used has to be present in the concept graph -- otherwise error will be given
concept = "/graphs/wikipedia/en-20120601/concepts/Differential_calculus"
url = 'https://gateway.watsonplatform.net/concept-insights/api'+\
      '/v2/corpora/liet4blerinu2/math_corp/documents/Calculus/'+\
    'relation_scores?concepts=["/graphs/wikipedia/en-20120601/concepts/"]'
r  = requests.get(url, auth=auth)
r.status_code

500

In [200]:
ast.literal_eval(r.content) # "/graphs/wikipedia/en-20120601/concepts/Radian"

{'error': 'internal error',
 'url': '/api/v2/corpora/liet4blerinu2/math_corp/documents/Calculus/relation_scores'}

##ConceptExpansion API

In [7]:
import json
import time
from watson_developer_cloud import ConceptExpansionV1Beta as ConceptExpansion


concept_expansion = ConceptExpansion(username='82a698f0-8d8c-42f9-9c7d-1e4b97eebd52',
                                     password='50KJG6jdgZ6h')

job_id = concept_expansion.create_job(dataset='mtsamples', 
                                      seeds=['motrin', 'tylenol', 'aspirin'], 
                                      label='medications')
print(json.dumps(job_id, indent=2))

time.sleep(5)  # sleep for 5 seconds
job_status = concept_expansion.get_status(job_id)

while job_status['status'] == 'in progress' or job_status['status'] == 'awaiting work':
    time.sleep(5)  # sleep for 5 seconds
    job_status = concept_expansion.get_status(job_id)
    print(json.dumps(job_status, indent=2))

if job_status['status'] == 'done':
    results = concept_expansion.get_results(job_id)
    print(json.dumps(results, indent=2))

{
  "jobid": "M0hUQkdKOTZJWkNDSUhRSVVCT047MTI0MzAxMTE="
}
{
  "return_seeds": [
    {
      "result": "ibuprofen", 
      "prevalence": 0.8219226002693176
    }, 
    {
      "result": "advil", 
      "prevalence": 0.80472731590271
    }, 
    {
      "result": "naproxen", 
      "prevalence": 0.7841069102287292
    }, 
    {
      "result": "benadryl", 
      "prevalence": 0.7685651183128357
    }, 
    {
      "result": "acetaminophen", 
      "prevalence": 0.7564734220504761
    }, 
    {
      "result": "codeine", 
      "prevalence": 0.743308424949646
    }, 
    {
      "result": "pain relievers", 
      "prevalence": 0.7278230786323547
    }, 
    {
      "result": "laxatives", 
      "prevalence": 0.7256090641021729
    }, 
    {
      "result": "motrin ib", 
      "prevalence": 0.7229762077331543
    }, 
    {
      "result": "aleve", 
      "prevalence": 0.7223989367485046
    }, 
    {
      "result": "diuretics", 
      "prevalence": 0.7221292853355408
    }, 
    {
      "